In [2]:
!pip install wordcloud

   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.2 kB ? eta -:--:--
   -- ------------------------------------ 20.5/300.2 kB 165.2 kB/s eta 0:00:02
   --- ----------------------------------- 30.7/300.2 kB 187.9 kB/s eta 0:00:02
   ---------- ---------------------------- 81.9/300.2 kB 383.3 kB/s eta 0:00:01
   ---------------------- --------------- 174.1/300.2 kB 655.4 kB/s eta 0:00:01
   ---------------------------------------  297.0/300.2 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- 300.2/300.2 kB 929.2 kB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from PIL import Image
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [4]:
 # importing our dataset
conversation ="WhatsApp Chat with Yashi.txt"

In [11]:
# now we will find time 
def date_time(s):
    pattern = r'^([0-9]+)(\/)([0-9]+)(\/)([0-9]+),([0-9]+)\s?(AM|PM|am|pm)?-'
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [12]:
# now finding the authors

def find_author(s):
    s = s.split(":")
    if len(s) == 2:
        return True
    else:
        return False

In [13]:
# now we will extraxt messages

def messages(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date,Time = dateTime.split(",")
    message=" ".join(splitline[1:])

    # now finding message belongs to which author
    if find_author(message):
        splitmessage = message.split(":")
        author = splitmessage[0]
        message=" ".join(splitmessage[1])
    else:
        author=None

    return date, Time, author, message

In [14]:
# Preparing our dataset now

data = []

conversation="WhatsApp Chat with Yashi.txt"

with open(conversation, encoding='utf-8') as fp:
    fp.readline()
    messageBuffer = []
    date, Time, author = None, None, None

    while True:
        line=fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                date.append([date, Time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, Time, author, message = messages(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [17]:
!pip install nltk

In [19]:
# now we will Train our dataset
# we will use sentiment intensity analyzer algorithm(SIAA), it is a part of NLTK VADER module which label dataset as positive or negative
# we use polarity score method to determine sentiment

df = pd.DataFrame(data, columns=["Date","Time","Author","Message"])
df['Date']=pd.to_datetime(df['Date'])
data = df.dropna()

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sentiment = SentimentIntensityAnalyzer()

data["Positive"] = [sentiments.polarity_score(i)['pos'] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_score(i)['neg'] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_score(i)['neu'] for i in data["Message"]]

print(data.head(10))

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...


Empty DataFrame
Columns: [Date, Time, Author, Message, Positive, Negative, Neutral]
Index: []


In [20]:
# validating our data

x = sum(data['Positive'])
y = sum(data['Negative'])
z = sum(data['Neutral'])

def score(a,b,c):
    if(a>b) and (a>c):
        print("Positive")
    elif(b>a) and (b>c):
        print("Negative")
    else:
        print("Neutral")

score(x,y,z)


Neutral


In [ ]:
# So we can see that the result is neutral i.e our whatsapp chat sentiment is neutral